<a href="https://colab.research.google.com/github/lekynam2000/EE4208/blob/main/PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!wget http://vis-www.cs.umass.edu/lfw/lfw.tgz
!tar zxvf lfw.tgz

In [2]:
%%capture
!pip install mtcnn tensorflow matplotlib sklearn numpy Pillow

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
data_path = "/content/gdrive/MyDrive/EE4208/Nam_working"
%cd $data_path

Mounted at /content/gdrive
/content/gdrive/.shortcut-targets-by-id/11EaBhGyyYl-1KsjTzeurrZifhefN88eu/EE4208/Nam_working


In [4]:
import pickle as pkl

with open("train_metadata.pkl",'rb') as f:
  train_metadata = pkl.load(f)
with open("test_metadata.pkl",'rb') as f:
  test_metadata = pkl.load(f)

In [5]:
!ls

Build_dataset.ipynb    PCA.ipynb	  train_metadata.pkl
FaceRecognition.ipynb  test_metadata.pkl


In [6]:
%cd /content
from PIL import Image
import numpy as np
for name,data_list in train_metadata.items():
  for data in data_list:
    img = Image.open(data["path"])
    # print(img.size)
    # print(data["box"])
    box = data['box']
    anchor = (box[0],box[1],box[0]+box[2],box[1]+box[3])
    img = img.crop(anchor).convert('L')
    img = np.array(img)
    img = img/255.0
    print(img)
    break
  break


/content
[[0.68235294 0.73333333 0.71764706 ... 0.28627451 0.38431373 0.38039216]
 [0.70196078 0.74901961 0.70588235 ... 0.2627451  0.33333333 0.36470588]
 [0.69411765 0.74117647 0.68627451 ... 0.23921569 0.27843137 0.35294118]
 ...
 [0.17647059 0.14509804 0.17647059 ... 0.65490196 0.54509804 0.49411765]
 [0.18431373 0.2        0.17254902 ... 0.68235294 0.52941176 0.45098039]
 [0.20392157 0.21176471 0.18039216 ... 0.69019608 0.4745098  0.39607843]]


In [7]:
from sklearn.preprocessing import LabelEncoder 
from sklearn.decomposition import PCA

le = LabelEncoder()
pca = PCA(n_components=0.9, svd_solver="full")


In [ ]:
def crop_and_grey(data):
  img = Image.open(data["path"])
  box = data['box']
  anchor = (box[0],box[1],box[0]+box[2],box[1]+box[3])
  img = img.crop(anchor).convert('L').resize((90,90))
  img = np.array(img)
  img = img/255.0
  img = np.reshape(img,(-1))
  img = np.expand_dims(img,axis=0)
  return img

In [10]:
X_train = None
y_train = []
for name,data_list in train_metadata.items():
  for data in data_list:
    img = crop_and_grey(img)
    if X_train is None:
      X_train = img
    else:
      X_train = np.vstack((X_train,img))
    y_train.append(name)
print(f"Before transform X.shape={X_train.shape}")
X_train = pca.fit_transform(X_train)
y_train = le.fit_transform(y_train)
print(f"After transform X.shape={X_train.shape}")

Before transform X.shape=(286, 8100)
After transform X.shape=(286, 66)


In [ ]:
X_test = None
y_test = []
for name,data_list in test_metadata.items():
  for data in data_list:
    img = crop_and_grey(img)
    if X_test is None:
      X_test = img
    else:
      X_test = np.vstack((X_test,img))
    y_test.append(name)

<h2>KNN Classification</h2>

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

<h3>Uniform weight </h3>